In [9]:
import pennylane as qml
from pennylane import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load iris data
iris = datasets.load_iris()
X = iris.data  # shape (150, 4)
y = iris.target  # 0,1,2

# Simple train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features (helps with small circuits)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Quantum setup
n_qubits = 4  # one qubit per feature (compact mapping)
dev = qml.device("default.qubit", wires=n_qubits)

# Simple feature map: angle encoding per feature
def feature_map(x):
    # x should be length 4
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

# Variational classifier: a single layer of rotations + a Z measurement
def circuit(params, x=None):
    # x: shape (4,)
    feature_map(x)
    # Parameterized layer (optional): apply RX and RZ with shared params
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    # Entangling layer (simple): ring of CNOTs
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])
    # Readout: expectations on PauliZ
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Define a QNode to produce a 4-dimensional quantum embedding from a single sample
@qml.qnode(dev)
def quantum_embedding(x, params):
    return circuit(params, x)

# Prepare a tiny dataset of embeddings
# We'll train a simple classical classifier on the flattened embedding
def make_embeddings(X, params):
    emb = []
    for x in X:
        emb.append(quantum_embedding(x, params))
    return np.array(emb)

# Initialize a small set of params
np.random.seed(0)
params = np.random.uniform(low=-0.5, high=0.5, size=(n_qubits,))

# Build a tiny pipeline: quantum embedding + classical logistic regression
# We'll use the embedding as features for Logistic Regression
class QuantumEmbedder:
    def __init__(self, params):
        self.params = params

    def transform(self, X):
        return make_embeddings(np.array(X), self.params)

# Train a logistic regression on the embeddings
embedder = QuantumEmbedder(params)
X_train_emb = embedder.transform(X_train_scaled)
X_test_emb  = embedder.transform(X_test_scaled)

clf = LogisticRegression(max_iter=200, multi_class="auto")
clf.fit(X_train_emb, y_train)

# Evaluate
y_pred = clf.predict(X_test_emb)
acc = accuracy_score(y_test, y_pred)

print(f"Test accuracy (simple quantum embedding with logistic classifier): {acc*100:.2f}%")


Test accuracy (simple quantum embedding with logistic classifier): 73.33%


/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
